In [1]:
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/home/nate/Bench/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

import pandas as pd
import pickle

# scrape leafly

## get strain names with 100+ reviews

In [10]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.leafly.com/explore')
try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[1]/div[1]/fieldset/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[2]/button').click()
except:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/fieldset/div/span[1]/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/div/button').click()

In [11]:
for i in range(7):
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div/div/div[2]/div[3]/button').click()
    
all_names = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div/div/div[2]').text

clean up all of the text and organize into:

'/type/name' where type is indica, sativa, or hybrid

In [21]:
name_string = all_names.replace('\n', ', ')
name_list = name_string.split(', ')

full_name_list = [x for x in name_list[::3]]
full_name_list = [x.replace(' ', '-') for x in full_name_list]

full_type_list = [y for y in name_list[2::3]]

In [50]:
link_list = []

for i, breed in enumerate(full_type_list):
    link = str('/'+breed+'/'+full_name_list[i]).lower()
    link_list.append(link)

In [52]:
len(link_list)

336

save this so we can skip this step if we need to come back

In [67]:
pickle_out = open('link_list.pkl', 'wb')
pickle.dump(link_list, pickle_out)
pickle_out.close()

## scrape strains

In [2]:
link_list = pd.read_pickle('link_list.pkl')

In [3]:
def strain_scrape(strain_link, num_pages):
    
    '''
    given a link, scrape the reviews from each page for the given number of pages
    '''
    
    reviews = []
    
    for page in range(num_pages):
        time.sleep(5)
        driver.get('https://www.leafly.com' + strain_link + '/reviews?page={}&sort=date'.format(page))
        
        for review in range(1,9):
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                reviews.append(review_text)
                driver.execute_script("window.history.go(-1)")
            except:
                try:
                    time.sleep(5)
                    driver.navigate.refresh()
                    driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/a'.format(review)).click()
                    review_text = driver.find_element_by_xpath('//*[@id="main"]/div/section/div[2]/div[2]/div[4]/div[1]').text
                    reviews.append(review_text)
                    driver.execute_script("window.history.go(-1)")
                except:
                    pass

    return reviews

the website has an initial popup that has one of two paths

then a second popup asking for an email address

this next cell sets up the page for our scrape

In [5]:
driver = webdriver.Chrome(chromedriver)
driver.get('https://www.leafly.com')
try:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[1]/div[1]/fieldset/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/div/form/div[2]/button').click()
except:
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/fieldset/div/span[1]/label').click()
    driver.find_element_by_xpath('/html/body/div[6]/div/div/form/div/button').click()

time.sleep(60)
driver.find_element_by_xpath('/html/body/div[9]/div/div/div/div[2]/form/div/a').click()

and this cell initiates it

In [ ]:
all_reviews = []

for link in link_list[54:]:
    try:
        strain_reviews = strain_scrape(link, 13)
        all_reviews.append(strain_reviews)
    except:
        pass

In [7]:
len(all_reviews)

45

In [10]:
pickle_out = open('all_reviews.pkl', 'wb')
pickle.dump(check, pickle_out)
pickle_out.close()

In [4]:
check = pd.read_pickle('all_reviews.pkl')
len(check)

53

In [9]:
check = check + all_reviews

In [8]:
link_list[54]

'/hybrid/fire-og'